In [102]:
#Importation des librairies
from os import chdir
import os
os.chdir(r'C:\Users\ACER\Desktop\Cours numerique\Cours Python\Parcours Data analyst Openclassrooms\Projet 4\dataset_P4')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime 
import scipy.stats as st

In [103]:
#Conversion des datasets en dataframe 
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
transactions = pd.read_csv("transactions.csv")

In [104]:
#Nettoyage fichier customers

In [105]:
#Detection valeurs non connues
customers.isna().sum()

client_id    0
sex          0
birth        0
dtype: int64

In [106]:
#Detection valeurs nulles
customers.isnull().sum()

client_id    0
sex          0
birth        0
dtype: int64

In [107]:
#convertir la colonne birth en date et créer une colonne birth avec l'année
customers["birth"] = pd.to_datetime(customers["birth"],format="%Y")
customers["birth_year"] = customers['birth'].dt.year

In [108]:
#création d'une variable age quantitatives basée sur la colonne birth 
now = pd.to_datetime('now')
customers['age'] = (now.year - customers['birth'].dt.year) - ((now.month - customers['birth'].dt.month))

#discrétisation de la variable age en une variable qualitatives "tranches_age" avec 8 classes
tranches_age = [(customers.age<=20),(customers.age>20)&(customers.age<=30),(customers.age>30)&(customers.age<=40),(customers.age>40)&(customers.age<=50),(customers.age>50)&(customers.age<=60),(customers.age>60)&(customers.age<=70),(customers.age>70)&(customers.age<=80),(customers.age>80)]
age_list = ["- 20 ans","21-30","31-40","41-50","51-60","61-70","71-80","> 80 ans"]
customers["cat_ages"] = np.select(tranches_age,age_list,default="Not specified")
customers["cat_ages"] = np.sort(customers["cat_ages"])
customers.head()

,client_id,sex,birth,birth_year,age,cat_ages
0,c_4410,f,1967-01-01,1967,52,- 20 ans
1,c_7839,f,1975-01-01,1975,44,- 20 ans
2,c_1699,f,1984-01-01,1984,35,- 20 ans
3,c_5961,f,1962-01-01,1962,57,- 20 ans
4,c_5320,m,1943-01-01,1943,76,- 20 ans


In [109]:
#Detection valeurs non connues
products.isna().sum()

id_prod    0
price      0
categ      0
dtype: int64

In [110]:
#Detection valeurs nulles
products.isnull().sum()

id_prod    0
price      0
categ      0
dtype: int64

In [111]:
#conversion de la variable categ en variable string
products["categ"] = products["categ"].apply(np.str)

In [112]:
#Nettoyage fichier transactions

In [113]:
#Detection valeurs non connues
transactions.isna().sum()

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

In [114]:
#Detection valeurs null
transactions.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

In [115]:
# jointure gauche entre les dataframes transactions et customers
data_livres = transactions.merge(customers,how='left',on=["client_id"])
data_livres.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
sex           0
birth         0
birth_year    0
age           0
cat_ages      0
dtype: int64

In [116]:
# jointure gauche entre les dataframes transactions et customers
data_livres = data_livres.merge(products,how='left',on=["id_prod"])
data_livres = data_livres.sort_values(by='date',ascending=True)
data_livres.isnull().sum()
#on note 103 observations sans valeur de prix ni categ

id_prod         0
date            0
session_id      0
client_id       0
sex             0
birth           0
birth_year      0
age             0
cat_ages        0
price         103
categ         103
dtype: int64

In [118]:
data_livres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337016 entries, 176588 to 38779
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     337016 non-null  object        
 1   date        337016 non-null  object        
 2   session_id  337016 non-null  object        
 3   client_id   337016 non-null  object        
 4   sex         337016 non-null  object        
 5   birth       337016 non-null  datetime64[ns]
 6   birth_year  337016 non-null  int64         
 7   age         337016 non-null  int64         
 8   cat_ages    337016 non-null  object        
 9   price       336913 non-null  float64       
 10  categ       336913 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 21.9+ MB


In [119]:
#creation d'une dataframe data_livres_error avec les transactions sans prix ni categ
data_livres_error = data_livres.loc[data_livres.price.isnull()]
data_livres_error

,id_prod,date,session_id,client_id,sex,birth,birth_year,age,cat_ages,price,categ
21078,0_2245,2021-03-01 00:09:29.301897,s_3,c_580,m,1988-01-01,1988,31,21-30,NaN,NaN
127490,0_2245,2021-03-02 15:55:49.728644,s_783,c_8060,f,1979-01-01,1979,40,61-70,NaN,NaN
114875,0_2245,2021-03-07 20:00:01.525768,s_3159,c_1189,m,1985-01-01,1985,34,21-30,NaN,NaN
163566,0_2245,2021-03-23 15:57:44.266387,s_10427,c_5869,f,1952-01-01,1952,67,31-40,NaN,NaN
277022,0_2245,2021-03-28 10:46:04.804784,s_12567,c_966,f,1981-01-01,1981,38,21-30,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
297223,0_2245,2022-02-20 03:50:42.646180,s_167926,c_7287,f,1978-01-01,1978,41,- 20 ans,NaN,NaN
37138,0_2245,2022-02-24 04:18:45.054650,s_169978,c_5953,m,1986-01-01,1986,33,21-30,NaN,NaN
24584,0_2245,2022-02-25 00:08:08.736068,s_170426,c_6236,f,1976-01-01,1976,43,71-80,NaN,NaN
124233,0_2245,2022-02-25 06:36:49.557289,s_170548,c_2242,f,1991-01-01,1991,28,31-40,NaN,NaN


In [120]:
#Il semble que le livre sous l'id "0_2245" ne s'est pas vu associé une categorie et un prix associé
data_livres_error.id_prod.describe()

count        103
unique         1
top       0_2245
freq         103
Name: id_prod, dtype: object

In [121]:
#recherche de la référence "0_2245" sur transactions
transactions.loc[transactions.id_prod == "0_2245"]

,id_prod,date,session_id,client_id
6235,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533
10802,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954
14051,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120
17486,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964
21078,0_2245,2021-03-01 00:09:29.301897,s_3,c_580
...,...,...,...,...
322710,0_2245,2021-04-06 19:59:19.462288,s_16936,c_4167
329417,0_2245,2021-03-30 23:29:02.347672,s_13738,c_7790
330490,0_2245,2021-12-03 14:14:40.444177,s_128815,c_6189
335531,0_2245,2021-04-27 18:58:47.703374,s_26624,c_1595


In [122]:
#recherche de la référence "0_2245" sur produits
products.loc[products.id_prod == "0_2245"]

,id_prod,price,categ


In [126]:
#On peut voir donc que les transactions sans prix ni categorie sont due au fait que le livre "0_2245" n'a pas été intégré dans la dataset products 
#Sachant que le livre référencé "0_2245" ne représente que 103 transactions sur les 337016 transactions, je vais donc supprimer les transactions liées à ce livre
data_livres = data_livres.loc[data_livres.id_prod != "0_2245"]

In [127]:
data_livres.loc[data_livres.id_prod == '0_2245']

,id_prod,date,session_id,client_id,sex,birth,birth_year,age,cat_ages,price,categ


In [128]:
data_livres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336913 entries, 176588 to 38779
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     336913 non-null  object        
 1   date        336913 non-null  object        
 2   session_id  336913 non-null  object        
 3   client_id   336913 non-null  object        
 4   sex         336913 non-null  object        
 5   birth       336913 non-null  datetime64[ns]
 6   birth_year  336913 non-null  int64         
 7   age         336913 non-null  int64         
 8   cat_ages    336913 non-null  object        
 9   price       336913 non-null  float64       
 10  categ       336913 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 21.8+ MB


In [129]:
#effacement des transactions test (id_prod = T_0)
data_livres = data_livres.loc[data_livres.id_prod != 'T_0']

In [130]:
data_livres.loc[data_livres.id_prod == 'T_0']

,id_prod,date,session_id,client_id,sex,birth,birth_year,age,cat_ages,price,categ


In [131]:
#transformer la colonne data en une colonne temporelle
data_livres["date"] = data_livres["date"].str[:-7]
data_livres["date"] = pd.to_datetime(data_livres["date"])

In [132]:
#focus analyse sur année 2021 : élimination des transactions ayant lieu en 2022
data_livres = data_livres.loc[data_livres['date'] < '2022-01-01']

In [133]:
#discrétisation de la variable date en déclinaisons de variables qualitatives pour les horaires, les jours, mois et années d'un achat + la tranche horaire correspondante
data_livres["time_purchase"] = pd.to_datetime(data_livres["date"], format='%H:%M:%S').dt.hour
data_livres["annee_purchase"] = pd.to_datetime(data_livres["date"], format='%y:%m:%d').dt.year
data_livres["month"] = pd.to_datetime(data_livres["date"], format='%y:%m:%d').dt.month
data_livres["day_purchase"] = pd.to_datetime(data_livres["date"], format='%y:%m:%d').dt.day

tranches_horaires = [(data_livres["time_purchase"]<=9),(data_livres["time_purchase"]>9)&(data_livres["time_purchase"]<12),(data_livres["time_purchase"]==12),(data_livres["time_purchase"]>12)&(data_livres["time_purchase"]<=18),(data_livres["time_purchase"]>18)]
tranches_list = ["night","morning","noon","afternoon","evening"]
data_livres["day_time_purchase"] = np.select(tranches_horaires,tranches_list,default='Not specificied')


In [134]:
data_livres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277846 entries, 176588 to 320212
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id_prod            277846 non-null  object        
 1   date               277846 non-null  datetime64[ns]
 2   session_id         277846 non-null  object        
 3   client_id          277846 non-null  object        
 4   sex                277846 non-null  object        
 5   birth              277846 non-null  datetime64[ns]
 6   birth_year         277846 non-null  int64         
 7   age                277846 non-null  int64         
 8   cat_ages           277846 non-null  object        
 9   price              277846 non-null  float64       
 10  categ              277846 non-null  object        
 11  time_purchase      277846 non-null  int64         
 12  annee_purchase     277846 non-null  int64         
 13  month              277846 non-null  int

In [100]:
# détection des tops clients
top_clients_ = data_livres.groupby('client_id').price.agg([len,sum]).sort_values(by="len",ascending=False)
top_clients_["percent"] = (top_clients_["sum"]/sum(top_clients_["sum"]))*100
top_clients_ = top_clients_.sort_values(by='percent',ascending=False)
top_clients_.head()

,len,sum,percent
client_id,,,
c_1609,10770.0,134404.46,2.835010
c_4958,2059.0,116371.64,2.454641
c_6714,3743.0,60520.41,1.276564
c_3454,2639.0,43584.00,0.919323
c_7421,173.0,2327.54,0.049095
